In [6]:
#!apt update
#!apt install unzip
!unzip ./opinrank+review+dataset.zip -d ./opinrankdataset

Archive:  ./opinrank+review+dataset.zip
 extracting: ./opinrankdataset/OpinRankDataset.pdf  
 extracting: ./opinrankdataset/OpinRankDataset.zip  


In [9]:
!unzip ./opinrankdataset/OpinRankDataset.zip -d ./opinrankdataset/OpinRankDataset

Archive:  ./opinrankdataset/OpinRankDataset.zip
  inflating: ./opinrankdataset/OpinRankDataset/OpinRankDataset.pdf  
   creating: ./opinrankdataset/OpinRankDataset/hotels/beijing/
  inflating: ./opinrankdataset/OpinRankDataset/hotels/beijing/china_beijing_aloft_beijing_haidian  
  inflating: ./opinrankdataset/OpinRankDataset/hotels/beijing/china_beijing_ascott_beijing  
  inflating: ./opinrankdataset/OpinRankDataset/hotels/beijing/china_beijing_autumn_garden_courtyard_hotel  
  inflating: ./opinrankdataset/OpinRankDataset/hotels/beijing/china_beijing_bamboo_garden_hotel  
  inflating: ./opinrankdataset/OpinRankDataset/hotels/beijing/china_beijing_beijing_century_towers  
  inflating: ./opinrankdataset/OpinRankDataset/hotels/beijing/china_beijing_beijing_dong_fang_hotel  
  inflating: ./opinrankdataset/OpinRankDataset/hotels/beijing/china_beijing_beijing_far_east_international_youth_hostel  
  inflating: ./opinrankdataset/OpinRankDataset/hotels/beijing/china_beijing_beijing_friendship_h

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
import pandas as pd
import os
import plotly.express as px
import plotly
import seaborn as sns
import numpy as np
import tqdm
import datetime

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', 5000)

import plotly.io as pio
pio.templates.default = 'simple_white'
from bertopic import BERTopic

## Loading data

In [14]:
def parse_file(file_name):
    tmp_data = []
    with open(file_name, 'rb') as f:
        for line in f:
            items = line.decode('ISO-8859-1').strip().split('\t', maxsplit=2)
            tmp_data.append(
                {
                    'date': items[0],
                    'title': items[1] if len(items) > 1 else '',
                    'body': items[2] if len(items) > 2 else ''
                }
            )
    tmp_df = pd.DataFrame(tmp_data)
    tmp_df['file_name'] = file_name
    return tmp_df

In [22]:
tmp_dfs = []

for city in ['london']:
    for hotel in tqdm.tqdm(os.listdir(f"./opinrankdataset/OpinRankDataset/hotels/{city}")):
        file_name = f"./opinrankdataset/OpinRankDataset/hotels/{city}/{hotel}"
        tmp_dfs.append(parse_file(file_name))

df = pd.concat(tmp_dfs)
df.sample(5)

100%|██████████| 875/875 [00:00<00:00, 2864.39it/s]


,date,title,body,file_name
68,Sep 18 2007,Not as advertised!,During my stay the security system was not working for all the rooms. I requested a room with two twin beds (available by request as advertised on expedia) but no extra bed was available. Staff are friendly but not helpful.,./opinrankdataset/OpinRankDataset/hotels/london/uk_england_london_the_sanctuary_house_hotel
158,Feb 1 2007,spacious rooms,"we stayed here the first of october for a week. Since we were celebrating a special anniversary we were upgraded to a jr. suite. What a lovely, spacious room it was too. Plus, shortly after we arrived in our room a complimentary bottle of champagne and a bowl of gourmet olives was delivered to us. It was absolutely perfect. The hotel is in a great location also. Really close to shopping on oxford street and close enough to walk to covent gardens. We weren't crazy about the restaurant after a breakfast there but did have &quot;snacks&quot; from room service, which was fine. I only have 2 complaints and they are minimal. We didn't like any of the restaurants that the concierge recommended. In fact, we walked out of several places be cause we we the only people there and they didn't look very clean either. We got the feeling they (concierge &amp; restaurant) were working together. Usually at night we would stop at the honor bar for 2 glasses of ice to take to our room- quicker than room service and there was usually some employee(same one always) who would watch us like we were stealing something.Like you don't mind paying their high room rates but not a couple of bucks for a coke if you wanted. He just made us feel uncomfortable every night. But, all in all, we would stay there again because it really is a gorgeous hotel in a great location.",./opinrankdataset/OpinRankDataset/hotels/london/uk_england_london_the_soho_hotel
8,Dec 17 2008,you get what you pay for,"it is as everywhere else: you get what you pay for. want the cheapest option close to vibrant camden town? there must be some downsides, and there are surely a few:1. the cleaniness is not one of the strengths of this hotel. myself being a camping lover for many years, i am not too picky when it comes to cleaniness, but this hotel can almost compare to that worn down camping place back then in portugal! it surely needs renovation.2. the doorlocks are a joke. certainly originally designed for cat cages or something like this, but not for hotel rooms.but then again, you won't find anything for this price in this area!",./opinrankdataset/OpinRankDataset/hotels/london/uk_england_london_queens_hotel
74,Jan 13 2007,Fine for the Price,"My friend and I stayed in the Ambassador's Hotel during our vacation to London. Our room was VERY small but everything worked and was clean. The maid service was good and quick and breakfast was a decent variety of foods. The location was great, it's pretty close to two different Tube station's and lots of grocery stores, internet cafes, etc. I always felt safe walking back in the evening.",./opinrankdataset/OpinRankDataset/hotels/london/uk_england_london_ambassadors_hotel
3,Oct 2 2009,Good value hotel ideal for groups of lads,"stayed here for two nights with a group of friends. there was one couple in the group and they hated this hotel mostly because they were afraid to walk to hotel at night from tube. the rest of us found the hotel ok. its in decent location about five minutes walk from tube. its clean, rooms are ok staff seemed friendly and its cheap. all in all i had no complaints about this hotel",./opinrankdataset/OpinRankDataset/hotels/london/uk_england_london_travelodge_london_aldgate_east_hotel


In [ ]:
def get_hotel_chain(x):
    """Extract hotel chain from file name
    """
    if x.startswith('london_holiday_inn'):
        return 'Holiday Inn'
    if x.startswith('london_park_plaza'):
        return 'Park Plaza'
    if x.startswith('london_millennium'):
        return 'Millemiun'
    if x.startswith('london_park_inn'):
        return 'Park Inn'
    if x.startswith('london_hilton'):
        return 'Hilton'
    if x.startswith('london_radisson'):
        return 'Radisson'
    if x.startswith('london_travelodge'):
        return 'Travelodge'
    
    return 'other'

In [32]:
for p in ['title', 'body']:
    df[p] = df[p].str.strip()

# Extract country, city, hotel chain and hotel name from file name
df['country'] = df.file_name.apply(lambda x: x.split('/')[-1].split('_', 2)[0])
df['city'] = df.file_name.apply(lambda x: x.split('/')[-1].split('_', 2)[1])
df['full_hotel'] = df.file_name.apply(lambda x: x.split('/')[-1].split('_', 2)[2])
df['hotel'] = df.full_hotel.apply(lambda x: get_hotel_chain(x))
df.sample(3)

,date,title,body,file_name,country,city,full_hotel,hotel
13,Sep 6 2007,Clean but pricey,The staff are very friendly at the Beaver hotel but the cost of staying here was too much.We stayed in a double en-suite room which was nice. The room was quite spacey and so was the bathroom.We had no problems at all with the room - though the hairdryer didn't work in the bathroom.All in all - I would recommend this hotel if you were only staying for one night as it is expensive.,./opinrankdataset/OpinRankDataset/hotels/london/uk_england_london_beaver_hotel,uk,england,london_beaver_hotel,other
18,Jul 5 2008,Ewcellent--- Couldnt have asked for more,"When I read the two review I was suprised to see only good reviews and I was a little anxious. The area is a little iffy and the out side of the Hotel looks like an indian restaurant but DO NOT LET THAT PUT YOU OFF.The Staff including the manager could not be anymore welcoming. one of the staff lead us to our room.. when we opened the door we were pleasantly suprised. The room was light and airy. The room was very spacious and extremely clean and well presented. the newly refurbished bathroom was really high spec. although our room was very close to the rail line we did not mind as we were so tired from sight seeing. The room was fully equiped with all mod cons including small fridge in which there was a small mini bar which was complimentry( included some light refreshments- chocalate,lemonade etc.) We had a look into the restaurant and I am diapointed now that we didnt try it as it looked amazing... It was asian cuisineandt the decor was fantastic. Also the rate we got was brilliant.A big pat on the back to the Staff and Management.",./opinrankdataset/OpinRankDataset/hotels/london/uk_england_london_pasha_hotel,uk,england,london_pasha_hotel,other
152,Dec 3 2004,RECOMMEND TEA AT THE RITZ,"Myself and a friend had afternoon tea,and we were not dissapointed. Our waiter was very attentive, asking us if we wanted anything more, and bringing out extra sandwichs, he even gave us extra chocolates for some friends who couldn't make the tea. The setting was wonderful. I would reccomend it to anyone who is visiting LONDON.",./opinrankdataset/OpinRankDataset/hotels/london/uk_england_london_the_ritz_london,uk,england,london_the_ritz_london,other


In [33]:
# How many reviews do we have per hotel?
df.groupby(['hotel'], as_index = False).aggregate({'full_hotel': ['count', 'nunique']})\
    .sort_values(('full_hotel', 'count'), ascending = False)

hotel full_hotel        
                    count nunique
7        other      66459     793
1  Holiday Inn       2844      26
0       Hilton       2671      11
6   Travelodge       2178      22
5     Radisson       2103      12
4   Park Plaza       1346       4
2    Millemiun       1209       5
3     Park Inn        539       2

In [35]:
# We have some reviews that do not belong to any of the 7 hotel chains. Let's remove them for now.
df = df[df.hotel != 'other']

In [37]:
# Which hotels have the most reviews?
df[df.country == 'uk'].hotel.value_counts().head(30)

hotel
Holiday Inn    2844
Hilton         2671
Travelodge     2178
Radisson       2103
Park Plaza     1346
Millemiun      1209
Park Inn        539
Name: count, dtype: int64

In [39]:
# How many duplicate reviews do we have?
df.shape[0], df.drop_duplicates().shape[0]

(12890, 12890)

In [40]:
df['id'] = list(range(df.shape[0]))
df['review'] = df.title + ' ' + df.body
df = df[['id', 'hotel', 'review']]

In [46]:
"""Let's visualize the number of reviews per hotel chain. We can see that the Holiday Inn and Park Plaza have the most reviews, while the Travelodge has the least.
"""

colormap = px.colors.qualitative.Safe

fig = px.bar(
    df.groupby('hotel')[['id']].count()\
        .sort_values('id', ascending = False),
    text_auto = 'd',
    title = 'Reviews for London Hotels',
    labels = {'value': 'number of reviews'}
)

fig.update_traces(marker_color=colormap[6],
                  marker_line_color=colormap[6],
                  marker_line_width=1.5,
                  opacity=0.9)

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False) 

fig.update_layout(showlegend = False)

## Translate Comments

In [49]:
import langdetect
import json
from deep_translator import GoogleTranslator

In [50]:
def detect_language(text):
    try:
        return langdetect.detect(text)
    except KeyboardInterrupt as e:
        raise(e)
    except:
        return '<-- ERROR -->'

def get_translation(text):
    try:
        return GoogleTranslator(source='auto', target='en').translate(str(text))
    except KeyboardInterrupt as e:
        raise(e)
    except:
        return '<-- ERROR -->'
    
tmp_data = []

for rec in tqdm.tqdm(df.to_dict('records')):
    tmp_data.append(
        {'id': rec['id'],
        'lang': detect_language(rec['review']),
        'reviews_transl': get_translation(rec['review'])}
    )

100%|██████████| 12890/12890 [43:57<00:00,  4.89it/s] 


In [51]:
df = df.merge(pd.DataFrame(tmp_data))

In [52]:
df.to_csv('raw_transl_data.csv', index = False, sep = '\t')

In [54]:
df[df.lang != 'en'].sample(5)

,id,hotel,review,lang,reviews_transl
9854,9854,Park Plaza,uno splendido albergo per una splendida vacanza,es,a splendid hotel for a splendid holiday
991,991,Holiday Inn,hotel très bien placé et calme,fr,hotel very well located and quiet
6842,6842,Park Plaza,Fantastic experience!,es,Fantastic experience!
3447,3447,Holiday Inn,Abgewrackt wirkendes Hotel,de,Drained looking hotel
2394,2394,Radisson,Lage perfekt sonst aber gar nix...,no,Location perfect but nothing else...


## Clearing not meaningful comments

In [57]:
df['reviews_transl'] = df.reviews_transl.str.strip()
df['reviews_len'] = df.reviews_transl.str.len()

In [ ]:
# レビューの文字数の分布を可視化. おおよそ 5 % のレビューが 20 文字以下あることがわかる。
fig = px.histogram(df,
                   x='reviews_len',
                   nbins=250,
                   range_x=[0, 500],
                   histnorm='percent',
                   labels={'reviews_len': 'number of characters', 'percent': 'share of reviews, %'},
                   title='Number of characters in reviews')

fig.update_traces(marker_color=colormap[6], opacity=0.9)
fig

In [ ]:
# レビューの文字数の分布をホテルチェーン別に可視化.
fig = px.histogram(df,
                   x='reviews_len',
                   color = 'hotel',
                   nbins=500,
                   range_x=[0, 1000],
                   labels={'reviews_len': 'number of characters', 'percent': 'share of reviews, %'},
                   title='Number of characters in reviews by hotel chain')
fig

In [ ]:
# レビューを頻度順にソートして、上位 20 件を表示.
df.reviews_transl.str.lower().str.strip().value_counts().head(20)

reviews_transl
<-- error -->                37
great hotel                  11
great location                8
perfect                       8
excellent hotel               7
good value for money          6
very good hotel               6
good hotel                    6
excellent value for money     5
very good                     5
very nice hotel               5
good value for london         4
very nice                     4
parfait                       4
great deal                    4
optimal                       4
great experience              3
excellent choice              3
perfect location              3
impeccable                    3
Name: count, dtype: int64

In [ ]:
# 20 文字以下のレビューは全体の約 4 % を占めている.
print(df[df.reviews_len < 20].shape[0], df.shape[0])
print(df[df.reviews_len < 20].shape[0]/df.shape[0])

517 12890
0.040108611326609776


In [ ]:
# 文字数が <= 20 のレビューと > 20 のレビューで、ホテルチェーンの分布を比較.
df['length_group'] = df.reviews_len.apply(lambda x: '<= 20' if x <= 20 else '> 20')

In [ ]:
# レビュー長ごとにホテルチェーンの分布を集計して、ピボットテーブルを作成.
len_stats_df = df.pivot_table(
    index = 'hotel',
    values = 'id',
    columns = 'length_group',
    aggfunc = 'count'
)

# レビュー長ごとのレビュー数を合計して、total 列を作成. その後、total 列の値でソート.
len_stats_df['total'] = len_stats_df.sum(axis = 1)
len_stats_df = len_stats_df.sort_values('total', ascending = False)
len_stats_df

length_group,<= 20,> 20,total
hotel,,,
Holiday Inn,161,2683,2844
Hilton,96,2575,2671
Travelodge,93,2085,2178
Radisson,81,2022,2103
Park Plaza,59,1287,1346
Millemiun,43,1166,1209
Park Inn,25,514,539


In [ ]:
# ホテルチェーンごとに、レビュー長グループの割合を可視化. 短いレビューの偏りは特に見られないことがわかる.
px.bar(
    len_stats_df.apply(lambda x: 100.*x/len_stats_df.total).drop('total', axis = 1),
    text_auto = '.2f',
    color_discrete_map = {
          '<= 20': colormap[2],
          '> 20': colormap[5]
      }, title = "Reviews' length by hotel",
    labels = {'value': 'share of reviews, %', 'course_id': 'course',
             'length_group': 'review length'}
)

In [77]:
df.sample(5)

,id,hotel,review,lang,reviews_transl,review_transl,reviews_len,length_group
1733,1733,Hilton,Gran elección,es,Great choice,Great choice,12.00,<= 20
7603,7603,Travelodge,"Go with low expectations and you will be fine We stayed at this hotel for one night in December as we needed somewhere close to Covent Garden for a party we were attending.The location is ideal for the covent garden area and for £105 per night (double room - room only) I guess you can't expect much in over priced London!!The staff were helpful on checking in but the room was at best &quot;average&quot;. When we arrived the room was freezing cold and was only heated by one small electric wall heater.......which even after 6 hours on full hardly heated the room at all. The bathroom was dated and the bath stained and it really needed all ripping out and starting again. The extraction fan didn't work in the bathroom so when you took a shower it steamed up completely. The next morning there was only &quot;luke warm&quot; water left by 11am, which didn't exactly make for a refreshing shower!The room did have a LCD TV and coffee / Tea making facilities and it was nice that we didn't have to check out until 12pm on the day of departure.While you don't expect the earth with a travel lodge this hotel needs a bit of TLC and money spending on it to bring it up to an acceptable modern standard. As a one night stay its fine but we couldn't really recommend it for much longer.",en,"Go with low expectations and you will be fine We stayed at this hotel for one night in December as we needed somewhere close to Covent Garden for a party we were attending.The location is ideal for the covent garden area and for £105 per night (double room - room only) I guess you can't expect much in over priced London!!The staff were helpful on checking in but the room was at best &quot;average&quot;. When we arrived the room was freezing cold and was only heated by one small electric wall heater.......which even after 6 hours on full hardly heated the room at all. The bathroom was dated and the bath stained and it really needed all ripping out and starting again. The extraction fan didn't work in the bathroom so when you took a shower it steamed up completely. The next morning there was only &quot;luke warm&quot; water left by 11am, which didn't exactly make for a refreshing shower!The room did have a LCD TV and coffee / Tea making facilities and it was nice that we didn't have to check out until 12pm on the day of departure.While you don't expect the earth with a travel lodge this hotel needs a bit of TLC and money spending on it to bring it up to an acceptable modern standard. As a one night stay its fine but we couldn't really recommend it for much longer.","Go with low expectations and you will be fine We stayed at this hotel for one night in December as we needed somewhere close to Covent Garden for a party we were attending.The location is ideal for the covent garden area and for £105 per night (double room - room only) I guess you can't expect much in over priced London!!The staff were helpful on checking in but the room was at best &quot;average&quot;. When we arrived the room was freezing cold and was only heated by one small electric wall heater.......which even after 6 hours on full hardly heated the room at all. The bathroom was dated and the bath stained and it really needed all ripping out and starting again. The extraction fan didn't work in the bathroom so when you took a shower it steamed up completely. The next morning there was only &quot;luke warm&quot; water left by 11am, which didn't exactly make for a refreshing shower!The room did have a LCD TV and coffee / Tea making facilities and it was nice that we didn't have to check out until 12pm on the day of departure.While you don't expect the earth with a travel lodge this hotel needs a bit of TLC and money spending on it to bring it up to an acceptable modern standard. As a one night stay its fine but we couldn't really reco

In [ ]:
# 20 文字以下のレビューをフィルタリングして、length_group 列を削除.
filt_df = df[df.reviews_len > 20].drop('length_group', axis = 1)

In [79]:
filt_df.sample(5)

,id,hotel,review,lang,reviews_transl,review_transl,reviews_len
1233,1233,Travelodge,good day badnight Very good hotel for location and transport links. easy to find but crossing the road to get to the hotel is pretty dreadful esp with a lot of luggage. it is also possible to walk to the hotel from all stations so a taxi may not be needed if with light luggage. The shops are great with a post office/cafe/subway/kfc and many more near by.the only problem that i had was sleeping at night time as my room was on the 2nd floor above the main tube tracks the nosie was so bad that i only had 3hrs sleep:-(. but this is mentioned on the website and when mentioned to the receptionist she was happy to move me to a quiet room. overall the hotel is well located and the security is ace( i had to use my swipe card 5 times to get to my room).,en,good day badnight Very good hotel for location and transport links. easy to find but crossing the road to get to the hotel is pretty dreadful esp with a lot of luggage. it is also possible to walk to the hotel from all stations so a taxi may not be needed if with light luggage. The shops are great with a post office/cafe/subway/kfc and many more near by.the only problem that i had was sleeping at night time as my room was on the 2nd floor above the main tube tracks the nosie was so bad that i only had 3hrs sleep:-(. but this is mentioned on the website and when mentioned to the receptionist she was happy to move me to a quiet room. overall the hotel is well located and the security is ace( i had to use my swipe card 5 times to get to my room).,good day badnight Very good hotel for location and transport links. easy to find but crossing the road to get to the hotel is pretty dreadful esp with a lot of luggage. it is also possible to walk to the hotel from all stations so a taxi may not be needed if with light luggage. The shops are great with a post office/cafe/subway/kfc and many more near by.the only problem that i had was sleeping at night time as my room was on the 2nd floor above the main tube tracks the nosie was so bad that i only had 3hrs sleep:-(. but this is mentioned on the website and when mentioned to the receptionist she was happy to move me to a quiet room. overall the hotel is well located and the security is ace( i had to use my swipe card 5 times to get to my room).,753.00
3930,3930,Park Plaza,"Good value for money! The good news is that this hotel has a great value for your money. I had on BAholidays which had a much better rate than expedia and other sites. Its in a good location from what I read about other hotels I was extremely satisfied with paying $150/night. The continental breakfast that was included was also very sufficient and served its purpose. If I were to be a bit picky, the towels weren't the cleaniest. The wireless internet was overpriced and I had trouble using it. I got logged out and could not log back in since I was not provided a username and password after paying 15 pounds.",en,"Good value for money! The good news is that this hotel has a great value for your money. I had on BAholidays which had a much better rate than expedia and other sites. Its in a good location from what I read about other hotels I was extremely satisfied with paying $150/night. The continental breakfast that was included was also very sufficient and served its purpose. If I were to be a bit picky, the towels weren't the cleaniest. The wireless internet was overpriced and I had trouble using it. I got logged out and could not log back in since I was not provided a username and password after paying 15 pounds.","Good value for money! The good news is that this hotel has a great value for your money. I had on BAholidays which had a much better rate than expedia and other sites. Its in a good location from what I read about other hotels I was extremely satisfied with paying $150/night. The continental breakfast that was included was also very sufficient and served its purpose. If I were to be a

In [ ]:
# レビューの言語の分布を可視化.
fig = px.bar(
    100 * filt_df.lang.value_counts(normalize = True).head(10),
    text_auto = '.2f',
    labels = {'value': 'share of reviews, %', 'index': 'language'},
    title = 'Top reviews languages'
)

fig.update_layout(showlegend = False)

fig.update_traces(marker_color=colormap[6],
                  marker_line_color=colormap[6],
                  marker_line_width=1.5,
                  opacity=0.9)

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False) 

In [ ]:
# 20 文字以下のレビューを除外したデータセットを保存.
filt_df.to_csv('hotel_reviews_with_transl.csv', index = False, sep = '\t')